# "Spikeet data validation"
> "Spikeet data validation tests"
- toc: true
- branch: master
- badges: true
- comments: true
- author: Sylvain Thibault
- categories: [AI, screener, EDA]

# Spikeet quote data validation using SP500 stocks with Yahoo and Cloudquant data.  
Using these data sources, we derive the top 20 daily stock by return close to close daily and intraday, and the bottom 20 by return close to close daily and intraday. We do this for the last 3 years.   
We compare the Yahoo results and the Cloudquant results and find them to be identical except for 2 instances with WBD and EXC. Yahoo is correct in both those cases.   
We then compare Yahoo results with the Spikeet data from 4 different screens as we could not figure out how to do OR logic and needed to run separate tests on Spikeet.  
We found many discrepancies between Yahoo results and Spikeet.  
Almost all Spikeet data was wrong in each of those cases except in one instance.  



In [1]:
import yfinance as yf
import pandas as pd
import os
import warnings
import liberator

In [2]:
# function to get sp500 tickers from scraping wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    table = pd.read_html(url, header=0)[0]
    tickers = table['Symbol'].tolist()
    return tickers

sp500_tickers = get_sp500_tickers()

## Yahoo finance data

In [3]:
# function get daily quotes from Yahoo for a list of tickers
def get_quotes(tickers):
    if os.path.exists('sp500_daily.csv'):
        data = pd.read_csv('sp500_daily.csv', index_col=[0], header=[0,1])
        return data
    
    # Fetch data for the last 3 years
    # end_date = pd.Timestamp.now()
    end_date = pd.Timestamp('2023-09-21')
    # start_date = end_date - pd.DateOffset(years=3)
    start_date = pd.Timestamp('2020-09-21')

    data = yf.download(tickers, start=start_date, end=end_date)
    data.to_csv('./sp500_daily.csv')
    return data

In [4]:
# clean column names
def clean_cols(name) -> str:
    return name.lower().replace(" ", "_").replace("%","")

In [5]:
# get quote data from Yahoo
df_yahoo = get_quotes(sp500_tickers)
df_yahoo.head()

Adj Close                                                  
                             A    AAL        AAPL       ABBV ABNB         ABT   
Date                                                                            
2020-09-28 00:00:00  97.644173  12.76  112.918945  76.775093  NaN   99.386688  \
2020-09-29 00:00:00  98.340843  12.25  112.064384  76.432030  NaN  100.100395   
2020-09-30 00:00:00  99.047356  12.29  113.753853  77.047775  NaN  103.564285   
2020-10-01 00:00:00  99.322098  12.58  114.716461  76.651939  NaN  103.383469   
2020-10-02 00:00:00  98.134804  13.00  111.013382  75.754715  NaN  101.347023   

                                                                    ...   
                          ACGL         ACN        ADBE         ADI  ...   
Date                                                                ...   
2020-09-28 00:00:00  29.260000  213.752274  488.510010  110.920738  ...  \
2020-09-29 00:00:00  28.870001  214.999588  489.329987  110.532234  ...   
2020-09-30 00:00:00  29.250000  216.832184  490.429993  110.617508  ...   
2020-10-01 00:00:00  29.400000  216.054993  499.510010  112.749519  ...   
2020-10-02 00:00:00  29.700001  213.387650  478.989990  108.030701  ...   

                      Volume                                                
                        WYNN      XEL       XOM     XRAY     XYL      YUM   
Date                                                                        
2020-09-28 00:00:00  3203500  2062300  23396500  1289000  806800  1694600  \
2020-09-29 00:00:00  4250900  2066200  20770500  1042800  885600  1612600   
2020-09-30 00:00:00  4450800  3208500  22223700  1159900  879900  1735200   
2020-10-01 00:00:00  4258900  2240200  36890200   671400  699000  1300700   
2020-10-02 00:00:00  3115800  2265300  32694300   654200  611400  2173300   

                                                        
                         ZBH    ZBRA     ZION      ZTS  
Date                                                    
2020-09-28 00:00:00   937197  272700  1478700  2293800  
2020-09-29 00:00:00   856445  190500  1437200  1046300  
2020-09-30 00:00:00  1040300  596400  1911600  1906700  
2020-10-01 00:00:00  1134957  461200  1515200  1061900  
2020-10-02 00:00:00  1000748  354500  3176100  1042600  

[5 rows x 3018 columns]

In [6]:
# flatten data and compute intraday and daily return for each ticker
df_yahoo_return = (
    df_yahoo
    .stack(level=1)
    .reset_index()
    .rename(columns={'level_1': 'symbol'})
    .rename(columns=clean_cols)
    .sort_values(by=['symbol', 'date'])
    .reset_index(drop=True)
    .groupby('symbol')
    .apply(lambda df_: df_.assign(intraday_return=(df_.close - df_.open)/df_.open,
                                  daily_return=df_.adj_close.pct_change()))
    .reset_index(drop=True)
    .assign(date=lambda df_: pd.to_datetime(df_.date))
    .round(2)
)
df_yahoo_return.head()

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
0,2020-09-28,A,97.64,99.51,99.73,98.12,98.64,1336600.0,0.01,NaN
1,2020-09-29,A,98.34,100.22,101.33,99.59,99.79,722400.0,0.00,0.01
2,2020-09-30,A,99.05,100.94,101.89,100.31,100.54,1362200.0,0.00,0.01
3,2020-10-01,A,99.32,101.22,102.41,100.54,101.77,723600.0,-0.01,0.00
4,2020-10-02,A,98.13,100.01,101.28,99.81,100.21,712200.0,-0.00,-0.01


In [7]:
# Get top 20 highest and lowest intraday movers
yahoo_top_20_intraday = df_yahoo_return.nlargest(20, 'intraday_return')
yahoo_bottom_20_intraday = df_yahoo_return.nsmallest(20, 'intraday_return')

# Get top 20 highest and lowest daily returns
yahoo_top_20_daily = df_yahoo_return.nlargest(20, 'daily_return')
yahoo_bottom_20_daily = df_yahoo_return.nsmallest(20, 'daily_return')

In [8]:
yahoo_top_20_daily

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
45969,2020-11-04,BIIB,355.63,355.63,363.92,251.75,253.20,14591500.0,0.40,0.44
46446,2022-09-28,BIIB,276.61,276.61,283.44,265.02,282.96,16232000.0,-0.02,0.40
62560,2020-11-09,CCL,19.25,19.25,19.86,17.71,18.49,185304300.0,0.04,0.39
46115,2021-06-07,BIIB,395.85,395.85,468.55,286.00,295.35,17954100.0,0.34,0.38
16554,2020-10-22,ALGN,453.23,453.23,459.00,424.00,424.32,5114700.0,0.07,0.35
292337,2020-11-09,REG,40.68,45.96,47.22,37.63,38.00,5281200.0,0.21,0.35
145170,2020-11-09,FRT,80.93,91.61,94.32,75.12,75.60,2783100.0,0.21,0.33
198145,2020-11-09,KIM,12.08,13.57,13.96,11.72,11.74,19590900.0,0.16,0.32
133860,2020-11-09,FANG,28.45,32.40,33.60,28.01,28.25,10946900.0,0.15,0.31
140397,2022-11-10,FICO,581.78,581.78,581.87,518.26,525.74,844600.0,0.11,0.31


## Cloudquant data liberator

In [9]:
df_liberator = liberator.get_dataframe(liberator.query(symbols = sp500_tickers, 
                                                   back_to = '2020-09-20', 
                                                   name='daily_bars_adjusted'))

In [10]:
df_liberator.head()

,_seq,_dsname,msg_len,msg,muts,timestamp,symbol,length,open,open_primary,...,vwap,dividend_amount,dividend_ratio,split_ratio,adjustment_factor,unadjusted_close,spread,bidvol,askvol,count
0,2444,adjusted_daily_oc_bar,74,b,1600695000000000,2020-09-21 09:30:00.000,A,23400,96.145203,96.145203,...,95.606796,0.0,1.0,1.0,0.979275,98.129997,0.059200,357103,173750,12763
1,26875,adjusted_daily_oc_bar,74,b,1600695000000000,2020-09-21 09:30:00.000,AAL,23400,12.820000,12.800000,...,12.355127,0.0,1.0,1.0,1.000000,12.210000,0.009493,20606866,23739436,212399
2,49075,adjusted_daily_oc_bar,74,b,1600695000000000,2020-09-21 09:30:00.000,AAPL,23400,102.683899,102.664299,...,105.046799,0.0,1.0,1.0,0.982246,110.080002,0.017400,64653941,71457320,1362791
3,84210,adjusted_daily_oc_bar,74,b,1600695000000000,2020-09-21 09:30:00.000,ABBV,23400,78.411201,78.508202,...,77.593697,0.0,1.0,1.0,0.879641,89.089996,0.014000,2540122,2128676,77606
4,141047,adjusted_daily_oc_bar,74,b,1600695000000000,2020-09-21 09:30:00.000,ABT,23400,100.680801,100.633400,...,100.401901,0.0,1.0,1.0,0.951615,105.949997,0.029300,1803448,1037149,48282


In [11]:
columns = ['timestamp','symbol','open','high','low','close','unadjusted_close']

In [12]:
df_liberator_return = (
 df_liberator
 [columns]
 .assign(date=lambda df_: pd.to_datetime(df_.timestamp).dt.date)
 .drop(columns=['timestamp'])
 .reindex(columns=['date', 'symbol', 'open', 'high', 'low', 'close', 'unadjusted_close'])
 .sort_values(by=['symbol', 'date'])
 .reset_index(drop=True)
 .groupby('symbol')
 .apply(lambda df_: df_.assign(intraday_return=(df_.close - df_.open)/df_.open,
                                daily_return=df_.close.pct_change()))
 .reset_index(drop=True)
 .round(2)
)
df_liberator_return.head()

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return
0,2020-09-21,A,96.15,96.52,94.50,96.10,98.13,-0.00,NaN
1,2020-09-22,A,95.81,96.92,95.47,96.58,98.62,0.01,0.00
2,2020-09-23,A,96.56,96.86,94.57,94.90,96.91,-0.02,-0.02
3,2020-09-24,A,94.57,94.96,93.46,94.34,96.34,-0.00,-0.01
4,2020-09-25,A,93.97,96.10,93.79,95.67,97.69,0.02,0.01


In [13]:
# Get top 20 highest and lowest intraday movers
cq_top_20_intraday = df_liberator_return.nlargest(20, 'intraday_return')
cq_bottom_20_intraday = df_liberator_return.nsmallest(20, 'intraday_return')

# Get top 20 highest and lowest daily returns
cq_top_20_daily = df_liberator_return.nlargest(20, 'daily_return')
cq_bottom_20_daily = df_liberator_return.nsmallest(20, 'daily_return')

## Spikeet data

In [14]:
# function to clean daily returns strings and convert to float
def clean_daily_returns(df_):
    return (df_
            .daily_return
            .str.replace(",","")
            .astype(float)
            .div(100)
            .round(2)
            )

#### Spikeet daily top 20

In [15]:
# [daily][0][close][adjusted] / [daily][-1][close][adjusted] > 1.2
df_skeet_daily_top = pd.read_csv('spikeet_daily_chg_gt_20.csv')
df_skeet_daily_top.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-CLOSE%
0,CRDF,2020-09-21,"206,217,480.95",8.90,11.87,8.86,11.60,"27,581.00",33.18
1,CLSK,2020-09-21,"200,747,131.20",11.00,15.07,10.73,14.84,"20,724.00",28.48
2,CRMD,2020-09-21,"144,009,918.34",5.23,6.06,4.85,5.72,"78,941.00",24.08
3,ACST,2020-09-21,"20,957,855.75",11.88,14.35,11.27,13.39,"84,294.21",28.99
4,ZOM,2020-09-21,"39,483,600.66",0.10,0.11,0.10,0.10,"8,517,670.00",42.86


In [16]:
df_skeet_daily_top.isna().sum()

Symbol                       4
Date                         0
Market Cap                2130
Open Price Today             0
High Price Today             0
Low Price Today              0
Close Price Today            0
Premarket Volume Today    1971
CHG-CLOSE%                   0
dtype: int64

- 4 symbols are NA in the data returned by Spikeet

In [17]:
df_skeet_daily_top.query('Symbol.isna()')

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-CLOSE%
7899,NaN,2022-07-25,NaN,7.10,13.00,7.01,10.56,133.00,55.52
7958,NaN,2022-07-29,NaN,7.81,12.49,7.35,9.62,"1,101.00",23.33
12693,NaN,2023-05-25,NaN,2.07,2.97,1.87,2.37,"2,113,598.00",63.45
12867,NaN,2023-06-08,NaN,2.10,2.95,2.10,2.72,"5,880.00",25.93


In [18]:
columns = ['date','symbol','open_price_today','high_price_today','low_price_today','close_price_today','daily_return']

In [19]:
skeet_top_20_daily = (
 df_skeet_daily_top
 .rename(columns=clean_cols)
 .rename(columns={'chg-close':'daily_return'})
 .assign(daily_return=lambda df_: clean_daily_returns(df_))
 .query('symbol in @sp500_tickers')
 .nlargest(25, 'daily_return')
 [columns]
 )
skeet_top_20_daily

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,daily_return
481,2020-11-04,BIIB,253.20,363.92,251.75,355.63,0.44
8776,2022-09-28,BIIB,282.96,283.44,265.02,276.61,0.40
535,2020-11-09,CCL,18.49,19.86,17.71,19.25,0.39
3930,2021-06-07,BIIB,295.35,468.55,286.00,395.85,0.38
392,2020-10-22,ALGN,424.32,459.00,424.00,453.23,0.35
532,2020-11-09,REG,38.00,47.22,37.63,45.96,0.35
645,2020-11-09,FRT,75.60,94.32,75.12,91.61,0.33
578,2020-11-09,KIM,11.74,13.96,11.72,13.57,0.32
684,2020-11-09,VLO,44.91,50.91,44.90,50.08,0.31
693,2020-11-09,FANG,28.25,33.60,28.01,32.40,0.31


#### Spikeet daily bottom 20

In [20]:
# [daily][0][close][adjusted] / [daily][-1][close][adjusted] < 0.8
df_skeet_daily_bottom = pd.read_csv('spikeet_daily_chg_lt_80.csv')
df_skeet_daily_bottom.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-CLOSE%,Close Price Yesterday
0,WVE,2020-09-21,"712,310,593.34",19.79,19.79,14.68,15.05,"2,318.00",-22.98,19.54
1,ACET,2020-09-21,"111,444,715.80",17.77,18.89,12.92,13.48,"42,030.00",-37.01,21.40
2,ICAD,2020-09-21,"265,646,867.20",10.50,10.50,8.70,9.26,"18,094.00",-20.17,11.60
3,BATL,2020-09-21,NaN,10.91,10.91,7.38,7.61,NaN,-29.99,10.87
4,CDOR,2020-09-21,"50,201,039.24",4.15,4.15,2.71,2.95,500.00,-29.43,4.18


In [21]:
df_skeet_daily_bottom.isna().sum()

Symbol                       3
Date                         0
Market Cap                1293
Open Price Today             0
High Price Today             0
Low Price Today              0
Close Price Today            0
Premarket Volume Today     883
CHG-CLOSE%                   0
Close Price Yesterday        0
dtype: int64

- 3 records with Symbol NA in returned Spikeet data

In [22]:
columns = ['date','symbol','open_price_today','high_price_today','low_price_today','close_price_today','close_price_yesterday','daily_return']

In [23]:
skeet_bottom_20_daily = (
 df_skeet_daily_bottom
 .rename(columns=clean_cols)
 .rename(columns={'chg-close':'daily_return'})
 .assign(daily_return=lambda df_: round(df_.daily_return/100,2))
 .query('symbol in @sp500_tickers')
 # get 25 to account for dup returns
 .nsmallest(25, 'daily_return')
 [columns]
 )
skeet_bottom_20_daily

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,close_price_yesterday,daily_return
5213,2023-02-01,C,51.54,52.74,51.41,52.15,261.10,-0.80
2320,2022-02-28,EPAM,351.44,355.76,198.25,207.75,382.43,-0.46
742,2021-03-03,GEN,0.73,0.77,0.45,0.50,0.81,-0.38
2698,2022-04-20,NFLX,245.20,248.70,212.51,226.19,348.61,-0.35
6910,2023-08-03,DXC,20.70,21.03,18.61,19.10,27.07,-0.29
4395,2022-11-03,FIS,67.53,68.36,56.53,57.18,79.47,-0.28
5592,2023-03-13,CMA,42.73,51.00,29.17,42.54,58.81,-0.28
6201,2023-05-04,PARA,18.91,19.04,16.39,16.40,22.89,-0.28
4317,2022-10-28,DVA,77.20,77.50,69.26,70.54,96.75,-0.27
5583,2023-03-13,KEY,12.24,12.89,9.61,11.38,15.66,-0.27


#### Spikeet intraday top 20

In [24]:
df_skeet_intra_top = pd.read_csv("./spikeet_intra_gt14.csv")
df_skeet_intra_top.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
0,NOG,2020-09-21,NaN,5.08,5.90,4.93,5.90,"11,389.00",16.14
1,CLSK,2020-09-21,"200,747,131.20",11.00,15.07,10.73,14.84,"20,724.00",34.91
2,CXDO,2020-09-21,"114,509,843.99",6.30,7.63,6.30,7.39,"1,100.00",17.30
3,SEED,2020-09-21,"48,513,488.32",9.09,10.86,8.69,10.70,442.00,17.71
4,BGMD,2020-09-21,NaN,0.01,0.01,0.01,0.01,NaN,17.89


In [25]:
columns = ['date','symbol','open_price_today','high_price_today','low_price_today','close_price_today','intra_return']

In [26]:
skeet_top_20_intra = (
 df_skeet_intra_top
 .rename(columns=clean_cols)
 .rename(columns={'chg-open':'intra_return'})
 .assign(intra_return=lambda df_ : (df_.intra_return.str.replace(",","")
                                    .astype(float))/100)
                                    .round(2)
 .query('symbol in @sp500_tickers')
 # get 25 to account for dup returns
 .nlargest(25, 'intra_return')
 [columns]
 )
skeet_top_20_intra

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
697,2020-11-04,BIIB,253.20,363.92,251.75,355.63,0.40
5544,2021-06-07,BIIB,295.35,468.55,286.00,395.85,0.34
19544,2023-03-13,RF,14.21,20.11,13.94,18.85,0.33
18495,2023-01-24,NCLH,12.35,18.54,12.35,15.43,0.25
9378,2022-02-24,ENPH,119.00,147.93,118.57,147.35,0.24
809,2020-11-09,FRT,75.60,94.32,75.12,91.61,0.21
823,2020-11-09,REG,38.00,47.22,37.63,45.96,0.21
8937,2022-02-24,SEDG,223.43,271.66,220.03,271.19,0.21
10068,2022-03-16,EPAM,233.51,284.50,232.30,281.72,0.21
11050,2022-05-04,MTCH,69.53,85.44,69.28,83.88,0.21


#### Spikeet intra bottom 20

In [27]:
df_skeet_intra_bottom = pd.read_csv("./spikeet_intra_bottom.csv")
df_skeet_intra_bottom.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
0,LBRT,2020-09-21,"764,526,989.62",8.83,8.83,7.43,7.65,240.00,-13.36
1,NTN,2020-09-21,"5,868,139.86",96.00,111.60,73.20,78.00,"57,602.88",-18.75
2,ACET,2020-09-21,"111,444,715.80",17.77,18.89,12.92,13.48,"42,030.00",-24.14
3,BATL,2020-09-21,NaN,10.91,10.91,7.38,7.61,NaN,-30.25
4,PIRS,2020-09-21,"154,575,870.00",2.84,2.86,2.24,2.33,"13,559.00",-17.96


In [28]:
df_skeet_intra_bottom.isna().sum()

Symbol                      14
Date                         0
Market Cap                5625
Open Price Today             0
High Price Today             0
Low Price Today              0
Close Price Today            0
Premarket Volume Today    7259
CHG-OPEN%                    0
dtype: int64

- 14 Symbols NA in Spikeet data

In [29]:
skeet_bottom_20_intra = (
 df_skeet_intra_bottom
 .rename(columns=clean_cols)
 .rename(columns={'chg-open':'intra_return'})
 .assign(intra_return=lambda df_: round(df_.intra_return/100,2))
 .query('symbol in @sp500_tickers')
 # get 25 to account for dup returns
 .nsmallest(25, 'intra_return')
 [columns]
 )
skeet_bottom_20_intra

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
8701,2022-02-28,EPAM,351.44,355.76,198.25,207.75,-0.41
2634,2021-03-03,GEN,0.73,0.77,0.45,0.50,-0.31
1071,2020-12-01,MRNA,177.66,178.50,130.33,141.01,-0.21
346,2020-10-14,GEN,0.65,0.65,0.53,0.53,-0.19
15812,2022-09-30,CCL,8.45,8.51,7.01,7.03,-0.17
12898,2022-06-21,DVA,90.36,91.62,74.97,76.05,-0.16
714,2020-11-09,CTLT,120.48,120.48,99.70,101.91,-0.15
6371,2021-11-16,AXON,208.84,209.00,174.78,177.79,-0.15
10736,2022-05-05,AXON,116.76,116.81,96.32,99.51,-0.15
16929,2022-11-03,FIS,67.53,68.36,56.53,57.18,-0.15


## Data provider comparison

#### Yahoo and Cloudquant

##### top 20 daily

In [30]:
diff_yahoo_top = yahoo_top_20_daily[~yahoo_top_20_daily['symbol'].isin(cq_top_20_daily['symbol'])]
diff_cq_top = cq_top_20_daily[~cq_top_20_daily['symbol'].isin(yahoo_top_20_daily['symbol'])]

In [31]:
# symbols in Yahoo top 20 but not in cq top 20
diff_yahoo_top

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return


In [32]:
# symbols in CQ top 20 but not in Yahoo top 20
diff_cq_top

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return


No difference for top 20 daily between Yahoo and Clouquant

##### Bottom 20 daily

In [33]:
diff_yahoo_bottom = yahoo_bottom_20_daily[~yahoo_bottom_20_daily['symbol'].isin(cq_bottom_20_daily['symbol'])]
diff_cq_bottom = cq_bottom_20_daily[~cq_bottom_20_daily['symbol'].isin(yahoo_bottom_20_daily['symbol'])]

In [34]:
# symbols in Yahoo but not in Cloudquant
diff_yahoo_bottom

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
358029,2021-03-26,WBD,41.9,41.9,58.21,34.6,57.45,106071700.0,-0.27,-0.27


In [35]:
# symbols in Cloudquant but not in Yahoo
diff_cq_bottom

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return
132881,2022-02-02,EXC,38.76,40.68,38.52,40.46,42.86,0.04,-0.26


In [36]:
# Cloudquant
(df_liberator_return
 .astype({"date": str})
 .query('symbol=="EXC" and date=="2022-02-02"')
 )

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return
132881,2022-02-02,EXC,38.76,40.68,38.52,40.46,42.86,0.04,-0.26


In [37]:
# Yahoo
(df_yahoo_return
 .query('symbol=="EXC" and date=="2022-02-02"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
130400,2022-02-02,EXC,40.46,42.86,43.09,40.8,41.06,29696500.0,0.04,0.04


Clouquant has wrong open of 38.76 on 2022-02-02 should be 41.06, not 38.76

In [38]:
# 2021-03-26	WBD	
# Cloudquant
(df_liberator_return
 .astype({"date": str})
 .query('symbol=="WBD" and date.str.contains("2021-03")')
 )

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return


In [39]:
# Cloudquant
(df_liberator_return
 .astype({"date": str})
 .query('symbol=="WBD"')
 )

,date,symbol,open,high,low,close,unadjusted_close,intraday_return,daily_return
362760,2022-04-04,WBD,24.90,25.98,23.20,25.50,25.50,0.02,NaN
362761,2022-04-05,WBD,25.75,26.22,24.25,24.41,24.41,-0.05,-0.04
362762,2022-04-06,WBD,24.13,24.25,23.37,23.98,23.98,-0.01,-0.02
362763,2022-04-07,WBD,24.26,24.26,22.53,23.12,23.12,-0.05,-0.04
362764,2022-04-08,WBD,22.58,24.53,22.58,24.47,24.47,0.08,0.06
...,...,...,...,...,...,...,...,...,...
363130,2023-09-25,WBD,11.15,11.18,10.62,10.66,10.66,-0.04,-0.04
363131,2023-09-26,WBD,10.57,10.81,10.43,10.69,10.69,0.01,0.00
363132,2023-09-27,WBD,10.76,11.11,10.76,11.03,11.03,0.03,0.03
363133,2023-09-28,WBD,11.08,11.10,10.66,10.69,10.69,-0.04,-0.03


In [40]:
# Yahoo
(df_yahoo_return
 .query('symbol=="WBD" and date=="2021-03-26"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
358029,2021-03-26,WBD,41.9,41.9,58.21,34.6,57.45,106071700.0,-0.27,-0.27


Cloudquant does not have WBD data before April of 2022, maybe name change
* 2 differences between Yahoo and Cloudquant for bottom 20 daily, seems to be issues with Cloudquant data.

##### top 20 intra

In [41]:
# top intra returns
df_skeet_intra_top = pd.read_csv("./spikeet_intra_gt14.csv")
df_skeet_intra_top.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
0,NOG,2020-09-21,NaN,5.08,5.90,4.93,5.90,"11,389.00",16.14
1,CLSK,2020-09-21,"200,747,131.20",11.00,15.07,10.73,14.84,"20,724.00",34.91
2,CXDO,2020-09-21,"114,509,843.99",6.30,7.63,6.30,7.39,"1,100.00",17.30
3,SEED,2020-09-21,"48,513,488.32",9.09,10.86,8.69,10.70,442.00,17.71
4,BGMD,2020-09-21,NaN,0.01,0.01,0.01,0.01,NaN,17.89


In [42]:
df_skeet_intra_top.isna().sum()

Symbol                      13
Date                         0
Market Cap                4168
Open Price Today             0
High Price Today             0
Low Price Today              0
Close Price Today            0
Premarket Volume Today    6345
CHG-OPEN%                    0
dtype: int64

- 13 Symbol NA in Spikeet returned data

In [43]:
skeet_top_20_intra = (
 df_skeet_intra_top
 .rename(columns=clean_cols)
 .rename(columns={'chg-open':'intra_return'})
 .assign(intra_return=lambda df_ : (df_.intra_return.str.replace(",","")
                                    .astype(float))/100)
                                    .round(2)
 .query('symbol in @sp500_tickers')
 .nlargest(25, 'intra_return')
 [columns]
 )
skeet_top_20_intra

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
697,2020-11-04,BIIB,253.20,363.92,251.75,355.63,0.40
5544,2021-06-07,BIIB,295.35,468.55,286.00,395.85,0.34
19544,2023-03-13,RF,14.21,20.11,13.94,18.85,0.33
18495,2023-01-24,NCLH,12.35,18.54,12.35,15.43,0.25
9378,2022-02-24,ENPH,119.00,147.93,118.57,147.35,0.24
809,2020-11-09,FRT,75.60,94.32,75.12,91.61,0.21
823,2020-11-09,REG,38.00,47.22,37.63,45.96,0.21
8937,2022-02-24,SEDG,223.43,271.66,220.03,271.19,0.21
10068,2022-03-16,EPAM,233.51,284.50,232.30,281.72,0.21
11050,2022-05-04,MTCH,69.53,85.44,69.28,83.88,0.21


##### bottom 20 intra

In [44]:
# bottom intra returns
df_skeet_intra_bottom = pd.read_csv("./spikeet_intra_bottom.csv")
df_skeet_intra_bottom.head()

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
0,LBRT,2020-09-21,"764,526,989.62",8.83,8.83,7.43,7.65,240.00,-13.36
1,NTN,2020-09-21,"5,868,139.86",96.00,111.60,73.20,78.00,"57,602.88",-18.75
2,ACET,2020-09-21,"111,444,715.80",17.77,18.89,12.92,13.48,"42,030.00",-24.14
3,BATL,2020-09-21,NaN,10.91,10.91,7.38,7.61,NaN,-30.25
4,PIRS,2020-09-21,"154,575,870.00",2.84,2.86,2.24,2.33,"13,559.00",-17.96


In [45]:
df_skeet_intra_bottom.isna().sum()

Symbol                      14
Date                         0
Market Cap                5625
Open Price Today             0
High Price Today             0
Low Price Today              0
Close Price Today            0
Premarket Volume Today    7259
CHG-OPEN%                    0
dtype: int64

- 14 Symbol NA in returned data

In [46]:
skeet_bottom_20_intra = (
 df_skeet_intra_bottom
 .rename(columns=clean_cols)
 .rename(columns={'chg-open':'intra_return'})
 .assign(intra_return=lambda df_: round(df_.intra_return/100,2))
 .query('symbol in @sp500_tickers')
 .nsmallest(25, 'intra_return')
 [columns]
 )
skeet_bottom_20_intra

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
8701,2022-02-28,EPAM,351.44,355.76,198.25,207.75,-0.41
2634,2021-03-03,GEN,0.73,0.77,0.45,0.50,-0.31
1071,2020-12-01,MRNA,177.66,178.50,130.33,141.01,-0.21
346,2020-10-14,GEN,0.65,0.65,0.53,0.53,-0.19
15812,2022-09-30,CCL,8.45,8.51,7.01,7.03,-0.17
12898,2022-06-21,DVA,90.36,91.62,74.97,76.05,-0.16
714,2020-11-09,CTLT,120.48,120.48,99.70,101.91,-0.15
6371,2021-11-16,AXON,208.84,209.00,174.78,177.79,-0.15
10736,2022-05-05,AXON,116.76,116.81,96.32,99.51,-0.15
16929,2022-11-03,FIS,67.53,68.36,56.53,57.18,-0.15


### Spikeet and Yahoo comparison

#### Daily top 20

In [47]:
diff_yahoo_skeet = yahoo_top_20_daily[~yahoo_top_20_daily['symbol'].isin(skeet_top_20_daily['symbol'])]
diff_skeet = skeet_top_20_daily[~skeet_top_20_daily['symbol'].isin(yahoo_top_20_daily['symbol'])]

In [48]:
# symbols in Yahoo top 20 daily not in Spikeet top 20 daily
diff_yahoo_skeet

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return


In [49]:
# symbols in Spikeet top 20 daily not in Yahoo
diff_skeet

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,daily_return
542,2020-11-09,EXPE,119.20,130.57,119.19,128.64,0.25
632,2020-11-09,MTB,110.64,125.38,110.64,123.92,0.25
4945,2021-10-27,ENPH,205.50,225.40,204.04,216.22,0.25
6231,2022-03-16,EPAM,233.51,284.50,232.30,281.72,0.25
687,2020-11-09,ZION,35.00,40.90,34.96,39.58,0.24


In [50]:
# Yahoo
(df_yahoo_return
 .query('symbol=="EXPE" and date=="2020-11-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
131598,2020-11-09,EXPE,128.64,128.64,130.57,119.19,119.2,19306700.0,0.08,0.25


EXPE is the same data just multiple 0.25 return in top 20

In [51]:
# Yahoo
(df_yahoo_return
 .query('symbol=="MTB" and date=="2020-11-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
240469,2020-11-09,MTB,112.64,123.92,125.38,110.64,110.64,3062500.0,0.12,0.25


Same, multiple 0.25 daily return in top 20

In [52]:
# Yahoo
(df_yahoo_return
 .query('symbol=="ENPH" and date=="2021-10-27"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
120531,2021-10-27,ENPH,216.22,216.22,225.4,204.04,205.5,15168400.0,0.05,0.25


In [53]:
# Yahoo
(df_yahoo_return
 .query('symbol=="EPAM" and date=="2022-03-16"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
122135,2022-03-16,EPAM,281.72,281.72,284.5,232.3,233.51,3724600.0,0.21,0.25


In [54]:
# Yahoo
(df_yahoo_return
 .query('symbol=="ZION" and date=="2020-11-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
374523,2020-11-09,ZION,35.89,39.58,40.9,34.96,35.0,5289500.0,0.13,0.24


Daily top 20 same for Spikeet and Yahoo

#### Daily bottom 20

In [55]:
diff_bottom_yahoo_skeet = yahoo_bottom_20_daily[~yahoo_bottom_20_daily['symbol'].isin(skeet_bottom_20_daily['symbol'])]
diff_bottom_skeet = skeet_bottom_20_daily[~skeet_bottom_20_daily['symbol'].isin(yahoo_bottom_20_daily['symbol'])]

In [56]:
# symbols in Yahoo daily bottom 20 and not in Spikeet
diff_bottom_yahoo_skeet

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
45972,2020-11-09,BIIB,236.26,236.26,239.50,223.25,230.82,9252600.0,0.02,-0.28
358029,2021-03-26,WBD,41.90,41.90,58.21,34.60,57.45,106071700.0,-0.27,-0.27
224946,2022-02-03,META,237.76,237.76,248.00,235.75,244.65,188119900.0,-0.03,-0.26


In [57]:
# symbols in Spikeet daily bottom 20 and not in Yahoo
diff_bottom_skeet

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,close_price_yesterday,daily_return
5213,2023-02-01,C,51.54,52.74,51.41,52.15,261.10,-0.80
742,2021-03-03,GEN,0.73,0.77,0.45,0.50,0.81,-0.38
2151,2022-02-02,PYPL,139.89,139.90,129.01,132.57,175.80,-0.25
3048,2022-05-18,TGT,163.07,168.00,155.20,161.61,215.28,-0.25
183,2020-11-03,SEDG,213.09,217.97,201.56,206.37,267.72,-0.23
4088,2022-09-30,CCL,8.45,8.51,7.01,7.03,9.16,-0.23
2119,2022-01-27,TER,110.52,112.40,102.51,111.24,143.37,-0.22


In [58]:
# Yahoo
(df_yahoo_return
 .query('symbol=="BIIB" and date=="2020-11-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
45972,2020-11-09,BIIB,236.26,236.26,239.5,223.25,230.82,9252600.0,0.02,-0.28


In [59]:
skeet_bottom_20_daily

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,close_price_yesterday,daily_return
5213,2023-02-01,C,51.54,52.74,51.41,52.15,261.10,-0.80
2320,2022-02-28,EPAM,351.44,355.76,198.25,207.75,382.43,-0.46
742,2021-03-03,GEN,0.73,0.77,0.45,0.50,0.81,-0.38
2698,2022-04-20,NFLX,245.20,248.70,212.51,226.19,348.61,-0.35
6910,2023-08-03,DXC,20.70,21.03,18.61,19.10,27.07,-0.29
4395,2022-11-03,FIS,67.53,68.36,56.53,57.18,79.47,-0.28
5592,2023-03-13,CMA,42.73,51.00,29.17,42.54,58.81,-0.28
6201,2023-05-04,PARA,18.91,19.04,16.39,16.40,22.89,-0.28
4317,2022-10-28,DVA,77.20,77.50,69.26,70.54,96.75,-0.27
5583,2023-03-13,KEY,12.24,12.89,9.61,11.38,15.66,-0.27


Spikeet missed BIIB on 2020-11-09 down 28% validated on tradingview

In [60]:
# Yahoo
(df_yahoo_return
 .query('symbol=="WBD" and date=="2021-03-26"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
358029,2021-03-26,WBD,41.9,41.9,58.21,34.6,57.45,106071700.0,-0.27,-0.27


Spikeet missed WDB on 2021-03-26 -27% validated on tradingview

In [61]:
# Yahoo
(df_yahoo_return
 .query('symbol=="META" and date=="2022-02-03"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
224946,2022-02-03,META,237.76,237.76,248.0,235.75,244.65,188119900.0,-0.03,-0.26


Missed META -26% on 2022-02-03

In [62]:
# Yahoo
(df_yahoo_return
 .query('symbol=="C" and date=="2023-02-01"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
56334,2023-02-01,C,50.49,52.15,52.74,51.41,51.54,22111900.0,0.01,-0.0


In [63]:
(df_skeet_daily_bottom
 .astype({"Date": str})
 .query('Symbol=="C" and Date=="2023-02-01"')
 
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-CLOSE%,Close Price Yesterday
5213,C,2023-02-01,NaN,51.54,52.74,51.41,52.15,"51,447.00",-80.03,261.10


- Spikeet has previous day close for C at 261.10, actual close for 2023-01-31 is 52.22

In [64]:
# Yahoo
(df_yahoo_return
 .query('symbol=="GEN" and date=="2021-03-03"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
149966,2021-03-03,GEN,18.93,19.99,20.11,19.7,19.84,5990000.0,0.01,0.0


In [65]:
(df_skeet_daily_bottom
 .astype({"Date": str})
 .query('Symbol=="GEN" and Date=="2021-03-03"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-CLOSE%,Close Price Yesterday
742,GEN,2021-03-03,"89,720,095.69",0.73,0.77,0.45,0.50,"735,336.00",-38.09,0.81


- Not sure where Spikeet gets its GEN quotes, wrong ticker ?

##### intra top 20

In [66]:
diff_intra_yahoo_skeet = yahoo_top_20_intraday[~yahoo_top_20_intraday['symbol'].isin(skeet_top_20_intra['symbol'])]
diff_intra_skeet = skeet_top_20_intra[~skeet_top_20_intra['symbol'].isin(yahoo_top_20_intraday['symbol'])]

In [67]:
# symbol in Yahoo not in Spikeet data
diff_intra_yahoo_skeet

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
292083,2022-11-03,RCL,53.77,53.77,54.20,45.80,46.16,14431800.0,0.16,0.08
42527,2022-02-24,BBWI,51.46,53.14,53.68,45.88,46.12,10078000.0,0.15,0.06
110875,2022-05-26,DXC,34.16,34.16,34.28,29.51,29.65,5632700.0,0.15,0.16


In [68]:
# symbol in Spikeet not in Yahoo data
diff_intra_skeet

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
18495,2023-01-24,NCLH,12.35,18.54,12.35,15.43,0.25
1609,2020-12-09,CTRA,10.55,12.69,10.55,12.61,0.20
3476,2021-02-09,GEN,0.95,1.15,0.95,1.14,0.20
4119,2021-03-09,GEN,0.50,0.74,0.49,0.60,0.20
18474,2023-01-24,VICI,28.54,34.12,28.49,33.96,0.19
1539,2020-12-07,CTRA,7.60,9.10,7.55,8.99,0.18
1586,2020-12-08,CTRA,8.99,11.39,8.99,10.55,0.17
18480,2023-01-24,MPC,111.72,143.28,111.72,130.21,0.17


In [69]:
# Yahoo
(df_yahoo_return
 .query('symbol=="RCL" and date=="2022-11-03"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
292083,2022-11-03,RCL,53.77,53.77,54.2,45.8,46.16,14431800.0,0.16,0.08


In [70]:
# Spikeet
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="RCL" and Date=="2022-11-03"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
15713,RCL,2022-11-03,"12,672,338,864.90",46.16,54.20,45.80,53.77,"309,843.00",16.49


- no issue, multiple entries at 16%

In [71]:
# Yahoo
(df_yahoo_return
 .query('symbol=="BBWI" and date=="2022-02-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
42527,2022-02-24,BBWI,51.46,53.14,53.68,45.88,46.12,10078000.0,0.15,0.06


In [72]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="BBWI" and Date=="2022-02-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
9384,BBWI,2022-02-24,"13,825,349,552.00",46.12,53.68,45.88,53.14,"4,487.00",15.22


- no issue, multiple entries at 15%

In [73]:
# Yahoo
(df_yahoo_return
 .query('symbol=="DXC" and date=="2022-05-26"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
110875,2022-05-26,DXC,34.16,34.16,34.28,29.51,29.65,5632700.0,0.15,0.16


In [74]:

(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="DXC" and Date=="2022-05-26"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
11770,DXC,2022-05-26,"7,472,989,125.85",29.65,34.27,29.51,34.16,"1,199.00",15.21


- no issue

In [75]:
diff_intra_skeet

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
18495,2023-01-24,NCLH,12.35,18.54,12.35,15.43,0.25
1609,2020-12-09,CTRA,10.55,12.69,10.55,12.61,0.20
3476,2021-02-09,GEN,0.95,1.15,0.95,1.14,0.20
4119,2021-03-09,GEN,0.50,0.74,0.49,0.60,0.20
18474,2023-01-24,VICI,28.54,34.12,28.49,33.96,0.19
1539,2020-12-07,CTRA,7.60,9.10,7.55,8.99,0.18
1586,2020-12-08,CTRA,8.99,11.39,8.99,10.55,0.17
18480,2023-01-24,MPC,111.72,143.28,111.72,130.21,0.17


In [76]:
# Yahoo
(df_yahoo_return
 .query('symbol=="NCLH" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
244039,2023-01-24,NCLH,15.43,15.43,15.9,14.95,15.68,11905900.0,-0.02,-0.01


- Yahoo has wrong open for NCLH Spikeet is correct with 25% intra return

In [77]:
# Yahoo
(df_yahoo_return
 .query('symbol=="CTRA" and date=="2020-12-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
90903,2020-12-09,CTRA,13.95,16.77,16.91,16.39,16.56,6461800.0,0.01,0.02


In [78]:
diff_intra_skeet.iloc[1].to_frame()

,1609
date,2020-12-09
symbol,CTRA
open_price_today,10.55
high_price_today,12.69
low_price_today,10.55
close_price_today,12.61
intra_return,0.2


- Spikeet has wrong quotes for CTRA 2020-12-09

In [79]:
 (df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="CTRA" and Date=="2020-12-09"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
1609,CTRA,2020-12-09,"193,158,240.90",10.55,12.69,10.55,12.61,NaN,19.53


In [80]:
# Yahoo
(df_yahoo_return
 .query('symbol=="GEN" and date=="2021-02-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
149951,2021-02-09,GEN,20.75,22.04,22.06,20.48,20.58,8233100.0,0.07,0.07


In [81]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="GEN" and Date=="2021-02-09"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
3476,GEN,2021-02-09,"104,004,057.16",0.95,1.15,0.95,1.14,"361,119.00",19.69


- Spikeet GEN quote for 2021-02-09 are wrong

In [82]:
# Yahoo
(df_yahoo_return
 .query('symbol=="GEN" and date=="2021-03-09"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
149970,2021-03-09,GEN,20.39,21.53,21.83,20.98,21.08,6189900.0,0.02,0.03


In [83]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="GEN" and Date=="2021-03-09"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
4119,GEN,2021-03-09,"55,520,463.71",0.50,0.74,0.49,0.60,"226,137.00",19.82


- GEN quotes wrong in Spikeet

In [84]:
# Yahoo
(df_yahoo_return
 .query('symbol=="VICI" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
349441,2023-01-24,VICI,32.69,33.96,34.12,32.44,32.45,3698700.0,0.05,0.01


In [85]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="VICI" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
18474,VICI,2023-01-24,"982,678,627.68",28.54,34.12,28.49,33.96,"2,725.00",18.99


- Yahoo quotes validated for VICI 2023-01-24, Spikeet quotes wrong.

In [86]:
# Yahoo
(df_yahoo_return
 .query('symbol=="CTRA" and date=="2020-12-07"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
90901,2020-12-07,CTRA,13.75,16.53,16.55,16.06,16.08,8927900.0,0.03,0.01


In [87]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="CTRA" and Date=="2020-12-07"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
1539,CTRA,2020-12-07,"139,147,168.80",7.60,9.10,7.55,8.99,NaN,18.29


- Spikeet quotes for CTRA 2020-12-07 are wrong, intraday chg-open is wrong

In [88]:
# Yahoo
(df_yahoo_return
 .query('symbol=="CTRA" and date=="2020-12-08"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
90902,2020-12-08,CTRA,13.65,16.42,16.73,16.1,16.26,6743400.0,0.01,-0.01


In [89]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="CTRA" and Date=="2020-12-08"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
1586,CTRA,2020-12-08,"164,596,453.62",8.99,11.39,8.99,10.55,100.00,17.35


- Spikeet data for CTRA 2020-12-08 is wrong

In [90]:
# Yahoo
(df_yahoo_return
 .query('symbol=="MPC" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
234237,2023-01-24,MPC,127.92,130.21,130.74,124.2,124.2,3782200.0,0.05,0.03


In [91]:
(df_skeet_intra_top
 .astype({"Date": str})
 .query('Symbol=="MPC" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
18480,MPC,2023-01-24,"59,463,686,491.04",111.72,143.28,111.72,130.21,"3,780.00",16.55


- Spikeet open is wrong for MPC 2023-01-24

##### intra bottom 20

In [92]:
diff_intra_bottom_yahoo_skeet = yahoo_bottom_20_intraday[~yahoo_bottom_20_intraday['symbol'].isin(skeet_bottom_20_intra['symbol'])]
diff_intra_bottom_skeet = skeet_bottom_20_intra[~skeet_bottom_20_intra['symbol'].isin(yahoo_bottom_20_intraday['symbol'])]

In [93]:
# symbol in Yahoo but not in Spikeet
diff_intra_bottom_yahoo_skeet

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
266795,2021-03-26,PARA,44.60,48.23,66.27,39.81,65.75,216631000.0,-0.27,-0.27
358029,2021-03-26,WBD,41.90,41.90,58.21,34.60,57.45,106071700.0,-0.27,-0.27
354492,2022-02-28,VTRS,10.29,11.01,13.74,10.88,13.72,62957100.0,-0.20,-0.24
266793,2021-03-24,PARA,64.83,70.10,85.00,70.00,84.40,89815300.0,-0.17,-0.23


In [94]:
# symbol in Spikeet but not in Yahoo
diff_intra_bottom_skeet

,date,symbol,open_price_today,high_price_today,low_price_today,close_price_today,intra_return
2634,2021-03-03,GEN,0.73,0.77,0.45,0.50,-0.31
346,2020-10-14,GEN,0.65,0.65,0.53,0.53,-0.19
20190,2023-01-24,TGT,169.82,169.82,144.30,144.32,-0.15
20205,2023-01-24,LLY,354.17,384.44,302.14,302.14,-0.15
20217,2023-01-24,VZ,38.69,44.72,32.76,33.04,-0.15
20203,2023-01-24,MRO,32.54,32.54,23.92,27.85,-0.14
20219,2023-01-24,UNP,235.20,235.20,201.00,203.18,-0.14
521,2020-10-27,BEN,22.60,22.60,19.66,19.73,-0.13


In [95]:
# Yahoo
(df_yahoo_return
 .query('symbol=="PARA" and date=="2021-03-26"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
266795,2021-03-26,PARA,44.6,48.23,66.27,39.81,65.75,216631000.0,-0.27,-0.27


In [96]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="PARA" and Date=="2021-03-26"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%


- Yahoo quotes validated intra day return of -27% missed by Spikeet

In [97]:
# Yahoo
(df_yahoo_return
 .query('symbol=="WBD" and date=="2021-03-26"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
358029,2021-03-26,WBD,41.9,41.9,58.21,34.6,57.45,106071700.0,-0.27,-0.27


In [98]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="WBD" and Date=="2021-03-26"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%


- Yahoo quotes for WBD 2021-03-26 validated, Spikeet quotes wrong and large intra loser missed

In [99]:
# Yahoo
(df_yahoo_return
 .query('symbol=="VTRS" and date=="2022-02-28"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
354492,2022-02-28,VTRS,10.29,11.01,13.74,10.88,13.72,62957100.0,-0.2,-0.24


In [100]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="VTRS" and Date=="2022-02-28"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%


- Yahoo quotes validated, Spikeet quotes and chg-open are wrong

In [101]:
# Yahoo
(df_yahoo_return
 .query('symbol=="PARA" and date=="2021-03-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
266793,2021-03-24,PARA,64.83,70.1,85.0,70.0,84.4,89815300.0,-0.17,-0.23


In [102]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="PARA" and Date=="2021-03-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%


In [103]:
# Yahoo
(df_yahoo_return
 .query('symbol=="GEN" and date=="2021-03-03"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
149966,2021-03-03,GEN,18.93,19.99,20.11,19.7,19.84,5990000.0,0.01,0.0


In [104]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="GEN" and Date=="2021-03-03"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
2634,GEN,2021-03-03,"89,720,095.69",0.73,0.77,0.45,0.50,"735,336.00",-31.23


- Here Spikeet flag GEN for an intraday loss of -31.23% with the wrong quotes.  Wrong ticker ?

In [105]:
# Yahoo
(df_yahoo_return
 .query('symbol=="GEN" and date=="2020-10-14"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
149871,2020-10-14,GEN,20.17,21.57,21.66,21.25,21.38,3479300.0,0.01,0.01


In [106]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="GEN" and Date=="2020-10-14"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
346,GEN,2020-10-14,"57,780,244.50",0.65,0.65,0.53,0.53,"3,765,455.00",-19.05


- Spikeet wrong again on GEN, wrong ticker ?

In [107]:
# Yahoo
(df_yahoo_return
 .query('symbol=="TGT" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
328329,2023-01-24,TGT,160.07,163.55,169.82,163.26,169.82,1902000.0,-0.04,-0.01


In [108]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="TGT" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
20190,TGT,2023-01-24,"75,780,849,787.44",169.82,169.82,144.30,144.32,"9,455.00",-15.02


- Spikeet wrong close and chg-open for TGT

In [109]:
# Yahoo
(df_yahoo_return
 .query('symbol=="LLY" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
209355,2023-01-24,LLY,343.64,346.39,346.63,341.36,346.0,2242500.0,0.0,0.01


In [110]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="LLY" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
20205,LLY,2023-01-24,"325,160,379,159.00",354.17,384.44,302.14,302.14,"1,757.00",-14.69


- Spikeet wrong quotes for LLY on 2023-01-24, seems to be numerous issues with Spikeet data for 2023-01-24

In [111]:
# Yahoo
(df_yahoo_return
 .query('symbol=="VZ" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
355473,2023-01-24,VZ,39.06,40.42,40.73,38.41,38.69,32143900.0,0.04,0.02


In [112]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="VZ" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
20217,VZ,2023-01-24,"166,359,751,725.06",38.69,44.72,32.76,33.04,"1,334,647.00",-14.6


- VZ Spikeet quotes wrong for close and chg-open

In [113]:
# Yahoo
(df_yahoo_return
 .query('symbol=="MRO" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
237253,2023-01-24,MRO,27.52,27.85,29.85,27.25,29.85,8649500.0,-0.07,-0.01


In [114]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="MRO" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
20203,MRO,2023-01-24,"17,791,527,335.90",32.54,32.54,23.92,27.85,"62,573.00",-14.41


- Wrong Spikeet quotes for MRO

In [115]:
# Yahoo
(df_yahoo_return
 .query('symbol=="UNP" and date=="2023-01-24"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
344917,2023-01-24,UNP,199.29,203.18,207.0,201.0,205.0,4462600.0,-0.01,-0.03


In [116]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="UNP" and Date=="2023-01-24"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
20219,UNP,2023-01-24,"128,682,208,541.73",235.20,235.20,201.00,203.18,"11,019.00",-13.61


- wrong Spikeet quotes for UNP

In [117]:
# Yahoo
(df_yahoo_return
 .query('symbol=="BEN" and date=="2020-10-27"')
 )

,date,symbol,adj_close,close,high,low,open,volume,intraday_return,daily_return
44455,2020-10-27,BEN,17.57,19.73,22.6,19.66,22.6,11141900.0,-0.13,-0.14


In [118]:
# Spikeet
(df_skeet_intra_bottom
 .astype({"Date": str})
 .query('Symbol=="BEN" and Date=="2020-10-27"')
 )

,Symbol,Date,Market Cap,Open Price Today,High Price Today,Low Price Today,Close Price Today,Premarket Volume Today,CHG-OPEN%
521,BEN,2020-10-27,"11,311,049,225.84",22.60,22.60,19.66,19.73,286.00,-12.7


- BEN is fine, multiple stocks with -13% intra return